In [9]:
!python -m venv myenv

!source myenv/bin/activate

In [13]:
!pip install seal
!pip install python-dp

DEPRECATION: Loading egg at /home/jun/.pyenv/versions/3.11.8/lib/python3.11/site-packages/seal-4.0.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/jun/.pyenv/versions/3.11.8/lib/python3.11/site-packages/seal-4.0.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
# 必要なライブラリのインポート
import pandas as pd
from seal import EncryptionParameters, SEALContext, KeyGenerator, Encryptor, BatchEncoder, Plaintext, Ciphertext, scheme_type, CoeffModulus, Decryptor, Evaluator, PlainModulus
import numpy as np


In [4]:

# データセットの読み込み
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)

# 年齢データの抽出
ages = data['age'].dropna().tolist()


In [47]:
from seal import EncryptionParameters, SEALContext, KeyGenerator, Encryptor, Plaintext, Ciphertext, BatchEncoder, scheme_type

# 暗号化パラメータの設定
parms = EncryptionParameters(scheme_type.bfv)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(1032193)  # ここを適切な値に変更

context = SEALContext(parms)
keygen = KeyGenerator(context)
public_key = keygen.create_public_key()
secret_key = keygen.secret_key()

encoder = BatchEncoder(context)
encryptor = Encryptor(context, public_key)

# 年齢データの暗号化
encrypted_ages = []
for age in ages:
    # リストをnumpy配列に変換
    age_array = np.array([age], dtype=np.int64)
    plain = encoder.encode(age_array)
    encrypted = encryptor.encrypt(plain)
    encrypted_ages.append(encrypted)



In [42]:
from seal import Evaluator, Plaintext

evaluator = Evaluator(context)
encrypted_sum = Ciphertext()
evaluator.add_many(encrypted_ages, encrypted_sum)

# 平均の計算（クライアント側で復号化するため、ここでは平均の計算を完了しません）


TypeError: add_many(): incompatible function arguments. The following argument types are supported:
    1. (self: seal.Evaluator, arg0: list[seal.Ciphertext]) -> seal.Ciphertext

Invoked with: <seal.Evaluator object at 0x7b460f0d52f0>, [], <seal.Ciphertext object at 0x7b460f0d6b30>

In [24]:
decryptor = Decryptor(context, secret_key)
plain_result = Plaintext()
decryptor.decrypt(encrypted_sum, plain_result)
sum_result = encoder.decode_int32(plain_result)
average_age = sum_result / len(ages)

# PyDPを使用して差分プライバシを適用
from pydp.algorithms.laplacian import BoundedMean

dp_ages = BoundedMean(1.0, 0, 120)  # epsilon=1.0, 年齢の最小値0歳、最大値120歳
dp_ages.add_entry(average_age)
private_average_age = dp_ages.result()

print("Private average age:", private_average_age)
